In [55]:
import pandas as pd
import snowflake.connector

In [56]:
conn = snowflake.connector.connect(user = "mdimasaac", password = "Snowflake123", account = "vliobpf-yr09142",
                                    database = "store_it", schema = "store_schema")
cur = conn.cursor()

In [83]:
qdf = "select * from store_data"
cur.execute(qdf)
df = cur.fetch_pandas_all()
print(df.shape)
df.head(1)

(10000, 12)


,DATE,MONTH,YEAR,PRODUCT_ID,PRODUCT_NAME,CATEGORY,BUY_PRICE,SELL_PRICE,QUANTITY,SALES,PROFIT_PER_ITEM,PROFIT
0,2022-11-02,11,2022,OFF-ST-10000988,"Fellowes Folders, Blue",Storage,25.99,33.53,3,77.97,7.54,22.62


In [5]:
len(df["CATEGORY"].unique().tolist())

17

### skip this


In [3]:
qdflog = "select * from logistics"
cur.execute(qdflog)
dflog = cur.fetch_pandas_all()
print(dflog.shape)
dflog.head()

(1856, 7)


,PRODUCT_NAME,PRODUCT_ID,CATEGORY,BUY_PRICE,SELL_PRICE,STOCK,PROFIT_PER_ITEM
0,"Fellowes Folders, Blue",OFF-ST-10000988,Storage,25.99,28.59,3,2.60
1,"SanDisk Numeric Keypad, Bluetooth",TEC-AC-10004144,Accessories,55.99,68.87,15,12.88
2,"Avery Legal Exhibit Labels, 5000 Label Set",OFF-LA-10001915,Labels,11.99,14.75,7,2.76
3,"Fellowes Folders, Wire Frame",OFF-ST-10004550,Storage,24.99,29.24,13,4.25
4,"Memorex Memory Card, USB",TEC-AC-10004068,Accessories,101.99,128.51,6,26.52


In [22]:
dfcart = pd.DataFrame(columns=["Product Name","Price","Qty","Total"])
dfcart.to_csv("cart.csv",index = False)

In [23]:
cart = pd.read_csv("cart.csv")
cart.head()

,Product Name,Price,Qty,Total


In [19]:

dfcopy = dflog.copy()
dfcopy["STOCK"][dfcopy["PRODUCT_NAME"] == "Fellowes Folders, Blue"] = dfcopy["STOCK"][dfcopy["PRODUCT_NAME"] == "Fellowes Folders, Blue"] - 1

C:\Users\mdima\AppData\Local\Temp\ipykernel_6928\4046239720.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfcopy["STOCK"][dfcopy["PRODUCT_NAME"] == "Fellowes Folders, Blue"] = dfcopy["STOCK"][dfcopy["PRODUCT_NAME"] == "Fellowes Folders, Blue"] - 1


In [20]:
dfcopy["STOCK"][dfcopy["PRODUCT_NAME"] == "Fellowes Folders, Blue"].values[0]

2

In [31]:
df1 = df[df["PRODUCT_NAME"] == "Fellowes Folders, Blue"].head(1)
df1["QUANTITY"][df1["PRODUCT_NAME"]=="Fellowes Folders, Blue"] = df1["QUANTITY"][df1["PRODUCT_NAME"]=="Fellowes Folders, Blue"]-1
df1["SALES"]=df1["QUANTITY"]*df1["SELL_PRICE"]
df1["PROFIT"]=df1["PROFIT_PER_ITEM"]*df1["QUANTITY"]
df1

,DATE,MONTH,YEAR,PRODUCT_ID,PRODUCT_NAME,CATEGORY,BUY_PRICE,SELL_PRICE,QUANTITY,SALES,PROFIT_PER_ITEM,PROFIT
0,2022-10-02,October,2022,OFF-ST-10000988,"Fellowes Folders, Blue",Storage,25.99,28.59,2,57.18,2.6,5.2


In [38]:
conn = snowflake.connector.connect(user = "mdimasaac", password = "Snowflake123", account = "vliobpf-yr09142",
                                    database = "store_it", schema = "store_schema")
cur = conn.cursor()
qdf = pd.DataFrame()
for name in dfcopy["CATEGORY"].unique().tolist():
# name = 'Fellowes Folders, Blue'
    q_df = "SELECT * FROM STORE_DATA WHERE CATEGORY = '"+name+"' LIMIT 1"
    cur.execute(q_df)
    qdf1 = cur.fetch_pandas_all()
    qdf = pd.concat([qdf,qdf1], axis = 0)
qdf.head()

2023-04-07 21:39:55.700 INFO    snowflake.connector.connection: Snowflake Connector for Python Version: 3.0.2, Python Version: 3.8.0, Platform: Windows-10-10.0.19041-SP0
2023-04-07 21:39:55.703 INFO    snowflake.connector.connection: This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2023-04-07 21:39:57.154 INFO    snowflake.connector.cursor: query: [SELECT * FROM STORE_DATA WHERE CATEGORY = 'Storage' LIMIT 1]
2023-04-07 21:39:57.352 INFO    snowflake.connector.cursor: query execution done
2023-04-07 21:39:57.355 INFO    snowflake.connector.cursor: Number of results in first chunk: 1
2023-04-07 21:39:57.359 INFO    snowflake.connector.cursor: query: [SELECT * FROM STORE_DATA WHERE CATEGORY = 'Accessories' LIMIT 1]
2023-04-07 21:39:57.512 INFO    snowflake.connector.cursor: query execution done
2023-04

,DATE,MONTH,YEAR,PRODUCT_ID,PRODUCT_NAME,CATEGORY,BUY_PRICE,SELL_PRICE,QUANTITY,SALES,PROFIT_PER_ITEM,PROFIT
0,2022-10-02,October,2022,OFF-ST-10000988,"Fellowes Folders, Blue",Storage,25.99,28.59,3,77.97,2.60,7.80
0,2022-10-02,October,2022,TEC-AC-10004144,"SanDisk Numeric Keypad, Bluetooth",Accessories,55.99,68.87,7,391.93,12.88,90.16
0,2022-10-02,October,2022,OFF-LA-10001915,"Avery Legal Exhibit Labels, 5000 Label Set",Labels,11.99,14.75,3,35.97,2.76,8.28
0,2022-10-02,October,2022,TEC-PH-10003963,"Apple Signal Booster, Full Size",Phones,137.99,157.31,3,413.97,19.32,57.96
0,2020-05-08,May,2020,TEC-CO-10004325,"Canon Personal Copier, Laser",Copiers,142.99,174.45,3,428.97,31.46,94.38


----------

In [ ]:
df.to_csv("store_data_snowflake.csv")

### and start from this

In [58]:
import plotly.express as px


In [85]:
# df["Y-M"] = pd.to_datetime(df.YEAR.astype(str) + '-' + df.MONTH.astype(str))
# df['MONTH'] = pd.to_datetime(df['MONTH'], format='%m').dt.strftime('%B')

df['DATE'] = pd.to_datetime(df['DATE']).dt.strftime('%b %Y')
df = df.sort_values(by="DATE")
df['DATE'] = pd.to_datetime(df['DATE'], format='%b %Y')
df = df.reset_index(drop=True)
df.head(1)

,DATE,MONTH,YEAR,PRODUCT_ID,PRODUCT_NAME,CATEGORY,BUY_PRICE,SELL_PRICE,QUANTITY,SALES,PROFIT_PER_ITEM,PROFIT
0,2020-04-01,4,2020,OFF-AR-10001759,"Sanford Sketch Pad, Fluorescent",Art,46.99,54.51,5,234.95,7.52,37.6


In [ ]:
# profit sum, quantity sum, sales sum

In [11]:
df[["CATEGORY","PROFIT"]][(df["MONTH"] == "4") & (df["YEAR"] == 2019)].groupby(["CATEGORY"]).sum()["PROFIT"]

CATEGORY
Accessories     104.94
Appliances       27.70
Art             404.87
Binders         294.68
Bookcases      1096.42
Chairs          254.00
Copiers        1192.20
Envelopes        23.79
Fasteners        32.88
Furnishings      89.92
Labels           38.08
Machines        107.64
Paper            79.06
Phones           66.20
Storage         979.42
Supplies        150.01
Name: PROFIT, dtype: float64

In [60]:
import plotly.express as px
import plotly.graph_objects as go

data = []
month = ["4","5","6"]
year = [2019]
m_name = ["Apr","May","Jun"]
x = df["CATEGORY"].sort_values().unique().tolist()
for i in year:
    for j,k in list(zip(month,m_name)):
        data.append(go.Bar(name=k+" "+str(i), x=x,
             y=df[["CATEGORY","PROFIT"]][(df["MONTH"] == j) & (df["YEAR"] == i)].groupby(["CATEGORY"]).sum()["PROFIT"]))

fig = go.Figure(data=data)
fig.update_layout(barmode='group', template="plotly_dark")

fig.show()

In [61]:
sel = df[(df["CATEGORY"]=="Art") & (df["YEAR"]==2022) & (df["MONTH"]=="9")].sort_values(by="PROFIT", ascending = False)
top6 = sel.groupby(["PRODUCT_NAME"]).sum().sort_values(by="PROFIT", ascending = False).head(6).index.values.tolist()
prod = []
for i in sel["PRODUCT_NAME"]:
    if (i in top6):
        prod.append(i)
    else:
        prod.append("Others")
sel["PRODUCT_NAME"] = prod
selected = sel.groupby(["PRODUCT_NAME"]).sum().sort_values(by="PROFIT", ascending = False)
selected

,YEAR,BUY_PRICE,SELL_PRICE,QUANTITY,SALES,PROFIT_PER_ITEM,PROFIT
PRODUCT_NAME,,,,,,,
Others,40440,537.80,633.60,62,1659.38,95.80,303.81
"Stanley Canvas, Fluorescent",4044,91.98,118.66,9,413.91,26.68,120.06
"BIC Sketch Pad, Easy-Erase",4044,97.98,117.58,9,440.91,19.60,88.20
"Stanley Canvas, Easy-Erase",4044,99.98,123.98,7,349.93,24.00,84.00
"BIC Canvas, Easy-Erase",2022,53.99,69.11,3,161.97,15.12,45.36
"Stanley Canvas, Blue",2022,48.99,63.69,3,146.97,14.70,44.10
"Stanley Pencil Sharpener, Water Color",6066,74.97,93.72,7,174.93,18.75,43.75


In [14]:
selected.PROFIT.values.round(2).tolist()

[613.26, 85.8, 84.24, 84.24, 84.0, 79.38, 74.88]

In [62]:
import plotly.express as px
import plotly.graph_objects as go

sel = df[(df["CATEGORY"]=="Phones") & (df["YEAR"]==2021) & (df["MONTH"]=="7")].sort_values(by="PROFIT", ascending = False)
top = sel.groupby(["PRODUCT_NAME"]).sum().sort_values(by="PROFIT", ascending = False).head(10).index.values.tolist()
prod = []
for i in sel["PRODUCT_NAME"]:
    if (i in top):
        prod.append(i)
    else:
        prod.append("Others")
sel["PRODUCT_NAME"] = prod
selected = sel.groupby(["PRODUCT_NAME"]).sum().sort_values(by="PROFIT", ascending = False)
labels = selected.index.values.tolist()
pull = []
for i in range(len(labels)):
    pull.append(0.1)
fig2 = go.Figure(go.Pie(
                    labels = labels,
                    values = selected.PROFIT.values.round(2).tolist(),hovertemplate = "%{label}<br>%{percent} <extra></extra>",
                    hole=.7,
                    marker_line_color='white',
                    marker_line_width=1.25,
                    pull= pull,
                    textinfo="label+percent"
                ))
fig2.update_layout(
                    autosize=True,  # Automatically adjust size to fit entire layout
                    margin=dict(l=0, r=0, t=0, b=0),)  # Remove margin around chart
fig2.update_layout(showlegend=False, template="plotly_dark")
fig2.update_layout(
font=dict(
size=16,))

fig2.show()
# selected = plotly_events(fig)

In [82]:
selected["PROFIT_PER_ITEM"][selected.index.values == "Samsung Smart Phone, VoIP"].values.tolist()[0]

152.64

In [63]:
dfline = df[df["PRODUCT_NAME"] == "Binney & Smith Canvas, Easy-Erase"].sort_values(by="DATE", ascending = True)
dfline

,DATE,MONTH,YEAR,PRODUCT_ID,PRODUCT_NAME,CATEGORY,BUY_PRICE,SELL_PRICE,QUANTITY,SALES,PROFIT_PER_ITEM,PROFIT
782,2019-11-22,11,2019,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,65.51,2,103.98,13.52,27.04
3796,2021-04-21,4,2021,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,65.51,3,155.97,13.52,40.56
3813,2021-04-22,4,2021,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,65.51,2,103.98,13.52,27.04
3956,2021-05-11,5,2021,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,65.51,3,155.97,13.52,40.56
4686,2021-09-18,9,2021,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,65.51,4,207.96,13.52,54.08
5583,2022-01-14,1,2022,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,65.51,5,259.95,13.52,67.60
6719,2022-06-05,6,2022,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,65.51,3,155.97,13.52,40.56
7731,2022-10-11,10,2022,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,65.51,2,103.98,13.52,27.04
7943,2022-10-30,10,2022,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,65.51,9,467.91,13.52,121.68
9029,2023-02-12,2,2023,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,65.51,1,51.99,13.52,13.52


In [64]:
fig = px.line(dfline, x="DATE", y="PROFIT", title='profit')
fig.update_layout(template = "plotly_dark")
fig.show()

In [93]:
dfline["PROFIT"].tolist()

[15.6, 15.6, 23.4, 23.4, 31.2, 39.0, 23.4, 70.2, 15.6, 7.8, 7.8, 23.4]

______

In [65]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

In [81]:
ml = df[["DATE","PRODUCT_NAME","CATEGORY","QUANTITY","PROFIT","MONTH","YEAR"]]
mo = []
for i in ml["MONTH"]:
    if len(str(i)) == 1:
        i = "0"+str(i)
        mo.append(i)
    else:
        mo.append(i)
ml["MONTH"] = mo
ml["TIME"] = ml["YEAR"].astype(str) + "/" + ml["MONTH"].astype(str)
ml.head()

C:\Users\mdima\AppData\Local\Temp\ipykernel_10444\296959859.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\mdima\AppData\Local\Temp\ipykernel_10444\296959859.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,DATE,PRODUCT_NAME,CATEGORY,QUANTITY,PROFIT,MONTH,YEAR,TIME
0,2022-11-02,"Fellowes Folders, Blue",Storage,3,7.80,November,2022,2022/November
1,2022-11-02,"SanDisk Numeric Keypad, Bluetooth",Accessories,7,58.80,November,2022,2022/November
2,2022-11-02,"Avery Legal Exhibit Labels, 5000 Label Set",Labels,3,4.68,November,2022,2022/November
3,2022-11-02,"Fellowes Folders, Wire Frame",Storage,2,8.50,November,2022,2022/November
4,2022-11-02,"Memorex Memory Card, USB",Accessories,3,58.14,November,2022,2022/November


In [ ]:
# index = product_name, category

In [82]:
ml_sel = ml[ml["CATEGORY"]=="Accessories"]
ml_sel

,DATE,PRODUCT_NAME,CATEGORY,QUANTITY,PROFIT,MONTH,YEAR,TIME
1,2022-11-02,"SanDisk Numeric Keypad, Bluetooth",Accessories,7,58.80,November,2022,2022/November
4,2022-11-02,"Memorex Memory Card, USB",Accessories,3,58.14,November,2022,2022/November
16,2021-06-16,"Belkin Memory Card, USB",Accessories,6,110.40,June,2021,2021/June
23,2022-06-05,"Memorex Mouse, Bluetooth",Accessories,1,7.56,June,2022,2022/June
29,2023-03-09,"Belkin Mouse, Erganomic",Accessories,5,40.95,March,2023,2023/March
...,...,...,...,...,...,...,...,...
9926,2021-03-18,"SanDisk Keyboard, USB",Accessories,1,11.48,March,2021,2021/March
9961,2020-11-08,"Logitech Memory Card, USB",Accessories,2,56.16,November,2020,2020/November
9963,2020-11-08,"Enermax Keyboard, Erganomic",Accessories,5,109.35,November,2020,2020/November
9972,2022-05-11,"Logitech Mouse, USB",Accessories,2,15.08,May,2022,2022/May


In [68]:
data = pd.DataFrame()
data["profit"] = ml_sel["PROFIT"]
from datetime import datetime
month_now = datetime.now().month
year_now = datetime.now().year
today = datetime.today()
if (pd.to_datetime(ml_sel["DATE"]).max().month)-1 != 0:
    ml_sel[ml_sel["DATE"]]


# shift1 = ml_sel[ml_sel[""]]
# data_shift1 = ml_sel.shift()
# data["shifted_1"] = 

int

In [79]:
pd.to_datetime(ml_sel["DATE"]).dt.month

67      5
93      5
113     6
131     6
136     6
       ..
9878    4
9899    4
9915    4
9917    4
9936    4
Name: DATE, Length: 452, dtype: int64

In [45]:
ml_timely = ml_sel.pivot_table(columns="TIME", values="PROFIT", aggfunc="sum", index=["PRODUCT_NAME"]).fillna(0)


In [133]:
ml_timely.index.values.tolist()[:5]

['Belkin Flash Drive, Bluetooth',
 'Belkin Flash Drive, Erganomic',
 'Belkin Flash Drive, Programmable',
 'Belkin Flash Drive, USB',
 'Belkin Keyboard, Bluetooth']

In [44]:
ml_name = ml_timely.index.values.tolist()

In [42]:
X = ml_timely.drop(columns = "2023/03")
y = ml_timely["2023/03"]
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import numpy as np
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.4, random_state = 43)
num_train = X_train.select_dtypes(np.number)
num_test = X_test.select_dtypes(np.number)
scaler = MinMaxScaler().fit(num_train)
cols = scaler.get_feature_names_out(input_features = num_train.columns)
num_train_scaled = scaler.transform(num_train)
num_test_scaled = scaler.transform(num_test)
num_train_scaled = pd.DataFrame(num_train_scaled, columns = cols)
num_test_scaled = pd.DataFrame(num_test_scaled, columns = cols)
X_train_processed = num_train_scaled.copy()
X_test_processed = num_test_scaled.copy()
import plotly.graph_objs as go

xvalues = ml_name.index.values.tolist()
yvalues = y_test
trace1 = go.Scatter(x=xvalues,y=yvalues,
                    mode = "lines", name = "actual profit")
regr = LinearRegression()
regr.fit(X_train_processed, y_train)
pred_test = regr.predict(X_test_processed)
mean_score_test = np.mean(cross_val_score(regr_left, X_test_processed, y_test, cv = 5))


NameError: name 'ml_name' is not defined